In [1]:
#Shotout to https://dafriedman97.github.io/mlbook/content/c7/construction.html
#You can learn more in there, Seriously!!!

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def ReLU(x):
    return np.max(0,x)
def sigmoid(x):
    return 1/(1+np.exp(-x))
def linear(x):
    return x

activation_function = {
    "ReLU": ReLU,
    "sigmoid": sigmoid,
    "linear": linear
}

In [6]:
class FeedForward:
    
    def fit(self, X,Y, n_hidden, f1 = 'ReLU', f2 = 'linear', loss='RSS', lr = 1e-5, n_iter=5000, seed=None):
        self.X = X
        self.Y = Y.reshape(len(Y),-1)
        self.N = len(X)
        self.D_X = self.X.shape[1]
        self.D_Y = self.Y.shape[1]
        self.Xt = self.X.T
        self.Yt = self.Y.T
        self.D_h = n_hidden
        self.f1, self.f2 = f1, f2
        self.loss = loss
        self.lr = lr
        self.n_iter = n_iter
        self.seed = seed
        if seed is not None:
            np.random.seed(self.seed)
        #initiate weight
        self.W1 = np.random.randn(self.D_h, self.D_X)/5
        self.b1 = np.random.randn(self.D_h,1)/5
        self.W2 = np.random.randn(self.D_Y, self.D_h)/5
        self.b2 = np.random.randn(self.D_Y,1)/5
        
        #initiate output
        self.H1 = (self.W1 @ self.Xt) + self.b1
        self.Z1 = activation_function[self.f1](self.H1)
        self.H2 = (self.W2 @ self.Z1) + self.b2
        self.Yhatt = activation_function[self.f2](self.H2)
        
        #Fit Weights
        for i in range(self.n_iter):
            #YHat
            if self.loss == 'RSS':
                self.dL_dYhatt = -(self.Yt - self.Yhatt)
            elif self.loss == 'log':
                self.dL_dYhatt = (-(self.Yt/self.Yhatt) + (1-self.Yt)/(1-self.Yhatt))
            
            #H2
            if self.f2 == 'linear':
                self.dYhatt_dH2 = np.ones((self.D_Y, self.N))
            elif self.f2 == 'sigmoid':
                self.dYhatt_dH2 = sigmoid(self.H2) * (1 - sigmoid(self.H2))
            #b2
            self.dL_db2 = np.sum(self.dL_dH2, 1)
            
            #W2
            self.dL_dZ1 = np.tensordot(self.W2, self.dL_dH2, (0,0))
            
            # H1
            if self.f1 == 'ReLU':
                self.dL_dH1 = self.dL_dZ1 * np.maximum(self.H1, 0)
            elif self.f1 == 'linear':
                self.dL_dH1 = self.dL_dZ1
            
            #b1
            self.dL_db1 = np.sum(self.dL_dH1, self.Xt, (1,1))
            
            #W1
            self.dL_dW1 = np.tensordot(self.dL_dH1, self.Xt )
            
            #Update Weights
            self.W1 -= self.lr * self.self.dL_dW1
            self.b1 -= self.lr * self.self.dL_db1.reshape(-1,1)
            self.W2 -= self.lr * self.self.dL_dW2
            self.b2 -= self.lr * self.self.dL_db2.reshape(-1,1)
            
            #Update outputs
            self.H1 = (self.W1 @ self.Xt) + self.b1
            self.Z1 = activation_function[self.f1](self.H1)
            self.H2 = (self.W2 @ self.Z1) + self.b2
            self.Yhatt = activation_function[self.f2](self.H2)
    def predict(self, X_test):
        X_testt = X_test.T
        self.h1 = (self.W1 @ self.Xt) + self.b1
        self.z1 = activation_function[self.f1](self.H1)
        self.h2 = (self.W2 @ self.Z1) + self.b2
        self.Yhatt = activation_function[self.f2](self.H2)
        return self.Yhatt